# Neural Network Learning

In [1]:
import numpy as np

## 해석

### np.random.randn

numpy: [random.randn](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)

표준 정규 분포 랜덤 값 반환:

In [2]:
np.random.randn()

0.7281522720829854

In [3]:
# 2x3 크기
np.random.randn(2, 3)

array([[ 0.20105179, -0.23909872,  2.00316243],
       [-1.23705328,  0.93097848, -0.08544406]])

### np.zeros_like

numpy: [zeros_like](https://numpy.org/doc/stable/reference/generated/numpy.zeros_like.html)

In [4]:
x = np.array([[0, 1, 2, 3], [4, 5, 6, 7]])
np.zeros_like(x)

array([[0, 0, 0, 0],
       [0, 0, 0, 0]])

### np.argmax

numpy: [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html)

In [5]:
a = np.arange(6).reshape(2,3) + 10
a

array([[10, 11, 12],
       [13, 14, 15]])

In [6]:
# 축을 지정하지 않으면 flatten 배열을 만들어 계산한다.
np.argmax(a)

5

In [7]:
# axis=0 은 열에서 가장 큰 수를 가진 행을 찾는다.
np.argmax(a, axis=0)

array([1, 1, 1])

In [8]:
# axis=1 은 행에서 가장 큰 수를 가진 열을 찾는다.
np.argmax(a, axis=1)

array([2, 2])

### np.nditer

numpy: [nditer](https://numpy.org/doc/stable/reference/generated/numpy.nditer.html)

multi index: 행 마다 하나씩 원소 접근한다.

In [9]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7])
it = np.nditer(x, flags=['multi_index'], op_flags=['readonly'])

while not it.finished:
    idx = it.multi_index
    print(idx, x[idx])
    it.iternext()

(0,) 0
(1,) 1
(2,) 2
(3,) 3
(4,) 4
(5,) 5
(6,) 6
(7,) 7


In [10]:
x = np.array([[0, 1, 2, 3], [4, 5, 6, 7]])
it = np.nditer(x, flags=['multi_index'], op_flags=['readonly'])

while not it.finished:
    idx = it.multi_index
    print(idx, x[idx])
    it.iternext()

(0, 0) 0
(0, 1) 1
(0, 2) 2
(0, 3) 3
(1, 0) 4
(1, 1) 5
(1, 2) 6
(1, 3) 7


In [11]:
x = np.array([ [[0, 1], [2, 3]], [[4, 5], [6, 7]] ])
it = np.nditer(x, flags=['multi_index'], op_flags=['readonly'])

while not it.finished:
    idx = it.multi_index
    print(idx, x[idx])
    it.iternext()

(0, 0, 0) 0
(0, 0, 1) 1
(0, 1, 0) 2
(0, 1, 1) 3
(1, 0, 0) 4
(1, 0, 1) 5
(1, 1, 0) 6
(1, 1, 1) 7


### cross entroy error

#### 1차원 → 2차원 변환

In [12]:
t = np.array([0, 0, 1, 0, 0])
y = np.array([0.1, 0.05, 0.6, 0.2, 0.05])

print('  shape dim')
print('t:', t.shape, t.ndim, t)
print('y:', y.shape, y.ndim, y)

  shape dim
t: (5,) 1 [0 0 1 0 0]
y: (5,) 1 [0.1  0.05 0.6  0.2  0.05]


In [13]:
t = t.reshape(1, t.size)
y = y.reshape(1, y.size)

print('   shape dim')
print('t:', t.shape, t.ndim, t)
print('y:', y.shape, y.ndim, y)

   shape dim
t: (1, 5) 2 [[0 0 1 0 0]]
y: (1, 5) 2 [[0.1  0.05 0.6  0.2  0.05]]


#### one-hot encode → 인덱스 변환

In [14]:
t = np.array([[0, 0, 1, 0, 0], [0, 0, 0, 1, 0]])
y = np.array([[0.1, 0.05, 0.6, 0.2, 0.05], [0.1, 0.05, 0.2, 0.6, 0.05]])

t.argmax(axis=1)

array([2, 3])

#### Batch 크기

In [15]:
y = np.array([[0.1, 0.05, 0.6, 0.2, 0.05], [0.1, 0.05, 0.2, 0.6, 0.05]])
print('Shape:', y.shape)
print('Batch Size:', y.shape[0])

Shape: (2, 5)
Batch Size: 2


#### 2차원배열[1차원배열, 1차원배열]

In [16]:
t = np.array([2, 3])
y = np.array([[0.1, 0.05, 0.6, 0.2, 0.05], [0.1, 0.05, 0.2, 0.6, 0.05]])

0부터 n-1까지 수를 가지는 배열 =  
y의 행렬 수로 1차원 배열 생성:

In [17]:
r = np.arange(y.shape[0])
r

array([0, 1])

t의 원소 값을 인덱스로 사용하여  
y의 행마다 값을 추출:

`y[[0, 1, ... n-1], [a, b, ... z]]`: `[y[0, a], y[1, b], ... y[n-1, z]]`

In [18]:
y[r, t]

array([0.6, 0.6])

---

## Neural Network Gradient

In [19]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from neural_network.activation import softmax
from loss import sum_squares_error, cross_entropy_error
from differentiation import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

신경망 생성:

In [20]:
net = simpleNet()

무작위 가중치 매개변수 값 생성:

In [21]:
print(net.W)

[[ 1.03384912 -1.05056454  2.52924286]
 [ 0.39225465 -1.12675878 -1.42707192]]


예측:

In [22]:
p = net.predict(x)
print(p)

[[[  0.39225465  -1.12675878  -1.42707192]
  [  3.24446218  -5.48140542   0.77726998]]

 [[  6.09666972  -9.83605206   2.98161187]
  [  8.94887726 -14.19069871   5.18595376]]]


예측 결과의 최대값 인덱스:

In [23]:
np.argmax(p)

9

예시 입력 및 정답 생성

2x3 크기 매개변수 값:  
- x: 크기 2
- t: 크기 3

In [24]:
x = np.array([0.6, 0.9])
t = np.array([0, 0, 1])

손실함수를 생성:

함수의 모양을 `f(x)` 형태로 잡아준다.  
`numerical_gradient(f, x)`에서 `f(x)`를 사용하는 부분과 통일하기 위해서.

In [25]:
f = lambda w: net.loss(x, t)

수치미분 함수에 `(손실함수, 매개변수)`를 전달하여 **매개변수 값에 대한 손실함수의 기울기**를 구한다.

In [26]:
dW = numerical_gradient(f, net.W)

In [27]:
print(dW)

[[ 0.38709545  0.02824485 -0.4153403 ]
 [ 0.58064317  0.04236727 -0.62301044]]


---

## 2 Layer Net

In [28]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from neural_network.activation import sigmoid, softmax
from loss import cross_entropy_error
from differentiation import numerical_gradient

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [29]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [30]:
# 하이퍼파라미터
iters_num = 100  # 반복 횟수를 적절히 설정한다. 10000
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

In [ ]:
for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

In [ ]:
# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()